# Importing and Exploring Data
https://cccbdb.nist.gov/exp1x.asp

In [97]:
# store the data obtained from cccbdb
H2_geo = {
    'H1': [0.0, 0.0, 0.0],
    'H2': [0.0, 0.0, 0.7414]
}
H2O_geo = {
    'O1': [0.0, 0.0, 0.1173],
    'H2': [0.0, 0.7572, -0.4692],
    'H3': [0.0, -0.7572, -0.4692]
}
benzene_geo = {
    'C1': [0.0000, 1.3970, 0.0000],
    'C2': [1.2098, 0.6985, 0.0000],
    'C3': [1.2098, -0.6985, 0.0000],
    'C4': [0.0000, -1.3970, 0.0000],
    'C5': [-1.2098, -0.6985, 0.0000],
    'C6': [-1.2098, 0.6985, 0.0000],
    'H7': [0.0000, 2.4810, 0.0000],
    'H8': [2.1486, 1.2405, 0.0000],
    'H9': [2.1486, -1.2405, 0.0000],
    'H10': [0.0000, -2.4810, 0.0000],
    'H11': [-2.1486, -1.2405, 0.0000],
    'H12': [-2.1486, 1.2405, 0.0000]
}

In [98]:
H2_geo

{'H1': [0.0, 0.0, 0.0], 'H2': [0.0, 0.0, 0.7414]}

In [99]:
H2O_geo

{'O1': [0.0, 0.0, 0.1173],
 'H2': [0.0, 0.7572, -0.4692],
 'H3': [0.0, -0.7572, -0.4692]}

In [100]:
benzene_geo

{'C1': [0.0, 1.397, 0.0],
 'C2': [1.2098, 0.6985, 0.0],
 'C3': [1.2098, -0.6985, 0.0],
 'C4': [0.0, -1.397, 0.0],
 'C5': [-1.2098, -0.6985, 0.0],
 'C6': [-1.2098, 0.6985, 0.0],
 'H7': [0.0, 2.481, 0.0],
 'H8': [2.1486, 1.2405, 0.0],
 'H9': [2.1486, -1.2405, 0.0],
 'H10': [0.0, -2.481, 0.0],
 'H11': [-2.1486, -1.2405, 0.0],
 'H12': [-2.1486, 1.2405, 0.0]}

# Bond Length Calculation

In [101]:
import math
import warnings
def compute_bond_length(coord1, coord2):
    """
    Computes the bond length between two atoms.

    Parameters:
        coord1: 1D array-like, cartesian coordinates of the first atom in angstroms
        coord2: 1D array-like, cartesian coordinates of the second atom in angstroms, must be of the same length as coord1
        
    Returns:
        The Euclidean distance between coord1 and coord2
    """
    d = math.sqrt(sum([(comp1-comp2)**2 for comp1, comp2 in zip(coord1, coord2)]))
    if (d >= 2):
        warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
    return d

In [102]:
# test
print(compute_bond_length(H2O_geo['O1'], H2O_geo['H2']))
print(compute_bond_length([0, 0], [1, 1])) # should be sqrt(2)
print(compute_bond_length([0, 0, 0], [3, 3, 3])) # should print warning

0.9577755948028744
1.4142135623730951
5.196152422706632


C:\Users\repti\AppData\Local\Temp\ipykernel_29444\233551919.py:16: UserWarning: The bond length of 5.196152422706632 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")


# Bond Angle Calculation
*Note*: I am using the formula
$$ cos\theta = \frac{\vec{BA} \cdot \vec{BC}} {|\vec{BA}||\vec{BC}|} $$
in calculating the angle using B as the vertex, which gives consistent results with regard to whether the angle is acute. The formula given in the instructions yields 180 subtracting what is visually perceived as the angle.

In [103]:
import numpy as np
def compute_bond_angle(coord1, coord2, coord3, classify=True):
    """
    Computes the bond angle between three atoms.
    coord1 -- coord2
                   |_\\
                      \\
                        coord3

    Parameters:
        coord1, coord2, coord3 must be of the same length
        coord1: 1D array-like, cartesian coordinates of the first atom in angstroms
        coord2: 1D array-like, cartesian coordinates of the second atom in angstroms
        coord3: 1D array-like, cartesian coordinates of the third atom in angstroms
        classify = True: if set to True, prints the angle and classify it as acute, right or obtuse
    Returns:
        The Euclidean distance between coord1 and coord2
    """
    # put into arrays
    A = np.array(coord1)
    B = np.array(coord2)
    C = np.array(coord3)
    # calculate vectors
    vec1 = A - B
    vec2 = C - B
    
    numer = np.dot(vec1, vec2) # dot product
    denom = np.linalg.norm(vec1) * np.linalg.norm(vec2) # norms multiplied
    theta_rad = math.acos(numer / denom) # get theta in radians
    theta_deg = theta_rad * 180 / math.pi # convert to degrees
    if classify:
        if theta_deg < 90:
            print(f"Acute angle of {theta_deg:.2f}")
        elif theta_deg > 90:
            print(f"Obtuse angle of {theta_deg:.2f}")
        else:
            print(f"Right angle of {theta_deg:.2f}")
    return theta_deg

In [104]:
# tests
compute_bond_angle([0,0,0], [1,0,0], [1,1,0]) # should be right angle
compute_bond_angle([0,0], [1,0], [0.5, 1]) # should be acute
compute_bond_angle([0,0], [1,0], [3,1]); # should be obtuse

Right angle of 90.00
Acute angle of 63.43
Obtuse angle of 153.43


# Automating the Calculation of Unique Bond Lengths and Angles

## Calculating all unique bond lengths

In [105]:
def calculate_all_bond_lengths(molecule, unique=True, bonds=[]):
    """
    Calculates all unique bond lengths

    Parameters:
        molecule: a dictionary with atom names as keys and 1D array-like Cartesian coordinates as values
        unique: whether duplicate bond lengths are ignored
        bonds [list of sets of two atom names]: if supplied, only consider given bonds; otherwise treat any pair of atoms as a bond

    Returns:
        All unique bond lengths in a list
    """
    consider_bonds = False
    if len(bonds)!=0:
        consider_bonds = True
    keys = list(molecule.keys())
    bond_lengths = []
    for i in range(0, len(keys)): # loop over all atoms
        atom1 = keys[i]
        coord1 = molecule[atom1] # get the first coordinates
        for j in range(i+1, len(keys)): # loop over all atoms after the ith one
            atom2 = keys[j]
            coord2 = molecule[atom2] # get the second coordinates
            # if not bonded: ignore
            if consider_bonds and set([atom1, atom2]) not in bonds:
                continue
            bond_length = compute_bond_length(coord1, coord2) # compute bond length
            bond_lengths.append(np.round(bond_length, decimals=3)) # store in list
    if unique:
        bond_lengths = np.unique(bond_lengths)
    print(f"All {"unique " if unique else ""}bond lengths: {bond_lengths}")
    return bond_lengths

In [106]:
benzene_bonds = [
    set(['C1', 'C2']),
    set(['C2', 'C3']),
    set(['C3', 'C4']),
    set(['C4', 'C5']),
    set(['C5', 'C6']),
    set(['C6', 'C1']),
    set(['C1', 'H7']),
    set(['C2', 'H8']),
    set(['C3', 'H9']),
    set(['C4', "H10"]),
    set(['C5', 'H11']),
    set(['C6', 'H12']),
]

In [107]:
H2O_bonds = [set(['O1', 'H2']), set(['O1', 'H3'])]

In [108]:
# tests
calculate_all_bond_lengths(H2_geo)
calculate_all_bond_lengths(H2O_geo, bonds=H2O_bonds)
calculate_all_bond_lengths(benzene_geo, bonds=benzene_bonds);

All unique bond lengths: [0.741]
All unique bond lengths: [0.958]
All unique bond lengths: [1.084 1.397]


In [109]:
len(calculate_all_bond_lengths(benzene_geo, unique=False)) # should be 12 choose 2 = 12*11/2 = 66

All bond lengths: [np.float64(1.397), np.float64(2.42), np.float64(2.794), np.float64(2.42), np.float64(1.397), np.float64(1.084), np.float64(2.154), np.float64(3.402), np.float64(3.878), np.float64(3.402), np.float64(2.154), np.float64(1.397), np.float64(2.42), np.float64(2.794), np.float64(2.42), np.float64(2.154), np.float64(1.084), np.float64(2.154), np.float64(3.402), np.float64(3.878), np.float64(3.402), np.float64(1.397), np.float64(2.42), np.float64(2.794), np.float64(3.402), np.float64(2.154), np.float64(1.084), np.float64(2.154), np.float64(3.402), np.float64(3.878), np.float64(1.397), np.float64(2.42), np.float64(3.878), np.float64(3.402), np.float64(2.154), np.float64(1.084), np.float64(2.154), np.float64(3.402), np.float64(1.397), np.float64(3.402), np.float64(3.878), np.float64(3.402), np.float64(2.154), np.float64(1.084), np.float64(2.154), np.float64(2.154), np.float64(3.402), np.float64(3.878), np.float64(3.402), np.float64(2.154), np.float64(1.084), np.float64(2.481),

C:\Users\repti\AppData\Local\Temp\ipykernel_29444\233551919.py:16: UserWarning: The bond length of 2.4196562338481056 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_29444\233551919.py:16: UserWarning: The bond length of 2.794 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_29444\233551919.py:16: UserWarning: The bond length of 2.1542920438046465 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_29444\233551919.py:16: UserWarning: The bond length of 3.4018947970212134 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Lo

66

## Calculating all unique bond angles

In [110]:
def calculate_all_bond_angles(molecule, unique=True, bonds=[], ignore_order=True, debug=False):
    """
    Calculates all unique bond angles

    Parameters:
        molecule: a dictionary with atom names as keys and 1D array-like Cartesian coordinates as values
        unique = True: whether duplicate bond angles are ignored
        bonds [list of sets of two atom names]: if supplied, only consider given bonds; otherwise treat any pair of atoms as a bond
        ignore_order = True: if set to true, treat each three atoms consituting an angle as an unordered set and skip duplicate sets. If set to false, treat as an ordered set.
        debug = False: sets debug mode, which will print the angle sequence along with its magnitude in degrees

    Returns:
        All unique bond angles in a list
    """
    consider_bonds = False
    if len(bonds)!=0:
        consider_bonds = True
    calculated = set() # record of already calculated pairs to avoid duplicates
    bond_angles = []
    for atom1, coord1 in molecule.items():
        for atom2, coord2 in molecule.items():
            if (atom1 == atom2):
                continue # skip self calculation
            for atom3, coord3 in molecule.items():
                if atom1 == atom3 or atom2 == atom3:
                    continue # skip self calculation
                # if not two bonds: ignore
                if consider_bonds and (set([atom1, atom2]) not in bonds or set([atom2, atom3]) not in bonds):
                    continue
                if ignore_order:
                    this_angle = tuple(sorted([atom1, atom2, atom3])) # sorted so that same atoms produce the same sequence; cast to tuple so that it is hashable to store in a set
                else:
                    this_angle = (atom1, atom2, atom3) # different order considered non-duplicates
                if this_angle in calculated: # if already recorded
                    continue
                
                if debug:
                    print(this_angle)
                calculated.add(this_angle) # record this sequence
                bond_angle = compute_bond_angle(coord1, coord2, coord3, classify=False) # calculate bond angle
                bond_angles.append(np.round(bond_angle, decimals=3)) # store in list
    if unique:
        bond_angles = np.unique(bond_angles)
    print(f"All {"unique " if unique else ""}bond angles: {bond_angles}")
    return bond_angles

In [111]:
# tests
calculate_all_bond_angles(H2O_geo, bonds=H2O_bonds, ignore_order=False, debug=True);

('H2', 'O1', 'H3')
('H3', 'O1', 'H2')
All unique bond angles: [104.48]


In [112]:
calculate_all_bond_angles(H2_geo)
calculate_all_bond_angles(H2O_geo, bonds=H2O_bonds)
calculate_all_bond_angles(benzene_geo, bonds=benzene_bonds);

All unique bond angles: []
All unique bond angles: [104.48]
All unique bond angles: [119.998 119.999 120.    120.001]


In [113]:
len(calculate_all_bond_angles(benzene_geo, unique=False)) # should be 12 choose 3 = 12*11*10/(3*2) = 220

All bond angles: [np.float64(120.001), np.float64(90.002), np.float64(60.002), np.float64(30.001), np.float64(25.834), np.float64(120.0), np.float64(145.836), np.float64(99.169), np.float64(60.001), np.float64(20.833), np.float64(90.002), np.float64(60.001), np.float64(30.0), np.float64(9.167), np.float64(55.834), np.float64(149.998), np.float64(115.836), np.float64(69.168), np.float64(30.0), np.float64(59.999), np.float64(29.999), np.float64(0.0), np.float64(39.167), np.float64(85.834), np.float64(180.0), np.float64(85.834), np.float64(39.167), np.float64(29.999), np.float64(9.167), np.float64(30.0), np.float64(69.168), np.float64(115.836), np.float64(149.998), np.float64(55.834), np.float64(25.834), np.float64(20.833), np.float64(60.001), np.float64(99.169), np.float64(145.836), np.float64(120.0), np.float64(60.0), np.float64(30.0), np.float64(0.0), np.float64(30.0), np.float64(60.0), np.float64(94.166), np.float64(64.166), np.float64(34.166), np.float64(4.166), np.float64(80.833), n

220

In [114]:
len(calculate_all_bond_angles(benzene_geo, ignore_order=False, unique=False)) # should be 12 permutate 3 = 12*11*10 = 1320

All bond angles: [np.float64(120.001), np.float64(90.002), np.float64(60.002), np.float64(30.001), np.float64(25.834), np.float64(120.0), np.float64(145.836), np.float64(99.169), np.float64(60.001), np.float64(20.833), np.float64(29.999), np.float64(90.002), np.float64(60.001), np.float64(30.0), np.float64(9.167), np.float64(55.834), np.float64(149.998), np.float64(115.836), np.float64(69.168), np.float64(30.0), np.float64(29.999), np.float64(59.999), np.float64(59.999), np.float64(29.999), np.float64(0.0), np.float64(39.167), np.float64(85.834), np.float64(180.0), np.float64(85.834), np.float64(39.167), np.float64(30.0), np.float64(60.001), np.float64(90.002), np.float64(29.999), np.float64(9.167), np.float64(30.0), np.float64(69.168), np.float64(115.836), np.float64(149.998), np.float64(55.834), np.float64(30.001), np.float64(60.002), np.float64(90.002), np.float64(120.001), np.float64(25.834), np.float64(20.833), np.float64(60.001), np.float64(99.169), np.float64(145.836), np.float6

1320